In [104]:
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm
import pandas as pd 
import re
import seaborn as sns
import matplotlib.pyplot as plt
import json
import csv

In [105]:
def pages_to_scrape(pages):
    Lista_url= []
    for page in tqdm(range(1,pages+1), desc='Scraping pages'):

        search_url = f'https://www.domiporta.pl/mieszkanie/sprzedam/mazowieckie/warszawa?PageNumber={page}'
        response = requests.get(search_url)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        # Znajdź wszystkie elementy <article> o klasie "sneakpeak"
        sneakpeak_articles = soup.find_all('article', class_='sneakpeak')

# Przetwarzanie każdego elementu sneakpeak
        for sneakpeak_article in sneakpeak_articles:
            # Sprawdź, czy ten artykuł należy do <div class="promoted__content promoted__content--active js-selected-slider slick-initialized slick-slider slick-dotted">
            if not sneakpeak_article.find_parent('div', class_='selected-offers__slider'):
                # Jeśli nie należy, to możesz go przetwarzać
                href_value = sneakpeak_article.find('a')['href']
                prop_url = 'https://www.domiporta.pl' + href_value
                Lista_url.append(prop_url)
        time.sleep(10)

    print('Skończone pobieranie ogłoszeń')
    return Lista_url


In [7]:
pd.set_option('display.max_columns', None)

In [114]:
urls = pages_to_scrape(30)

Scraping pages: 100%|██████████| 30/30 [05:36<00:00, 11.21s/it]

Skończone pobieranie ogłoszeń


In [116]:
len(urls)

1080

In [122]:
urls[17]

'https://www.domiporta.pl/nieruchomosci/sprzedam-mieszkanie-czteropokojowe-warszawa-mokotow-granitowa-92m2/154474644'

In [123]:
wszystkie_mieszkanie_info = []

for link in tqdm(urls, desc="Processing links"):
    # Pobierz zawartość strony
    response = requests.get(link)
    html_content = response.text

    # Parsuj zawartość przy użyciu Beautiful Soup
    soup = BeautifulSoup(html_content, 'html.parser')

    mieszkanie_info = {'url': link}

    # Znajdź wszystkie wystąpienia skryptu
    scripts = soup.find_all('script')
    script_text = scripts[13].text
    keys_to_find = ['advertId', 'advertiserType', 'advertiserId', 'market']

    for key in keys_to_find:
        pattern = f"'{key}': '(.*?)'"
        match = re.search(pattern, script_text)
        if match:
            mieszkanie_info[key] = match.group(1)

    # Znajdź kontener zawierający informacje
    features_container = soup.find('div', class_= 'features__container')
    features_container_2 = soup.find('div', class_= 'features__container features__list-feature')

    # Funkcja do przetwarzania elementów
    def process_feature_elements(elements):
        current_key = None
        for element in elements:
            if 'features__item_name' in element['class']:
                # Jeśli to klucz, ustaw bieżący klucz
                current_key = element.text.strip().replace('\xa0', ' ')
            elif 'features__item_value' in element['class'] and current_key is not None:
                # Jeśli to wartość klucza, przypisz wartość do odpowiedniego klucza w słowniku
                mieszkanie_info[current_key] = element.text.strip().replace('\xa0', ' ')


    # Przetwarzanie pierwszego zestawu elementów
    try:
        feature_elements = features_container.find_all(['span', 'dt', 'dd'], class_=['features__item_name', 'features__item_value'])
        process_feature_elements(feature_elements)

    # Przetwarzanie drugiego zestawu elementów
        if features_container_2:
            feature_elements2 = features_container_2.find_all(['span', 'dt', 'dd'], class_=['features__item_name', 'features__item_value'])
            process_feature_elements(feature_elements2)
    except AttributeError:
    #W przypadku, gdy ogłoszenie nie aktywne, kontynnuj   
        continue     
    wszystkie_mieszkanie_info.append(mieszkanie_info)
time.sleep(5)


Processing links: 100%|██████████| 1080/1080 [11:38<00:00,  1.55it/s]


In [201]:
df = pd.DataFrame(wszystkie_mieszkanie_info)

In [171]:
df.head(5)

,url,advertId,advertiserType,advertiserId,market,Kategoria,Lokalizacja,Cena,Cena za m2,Typ budynku,Materiał,Forma własności,Liczba pokoi,Numer oferty,Piętro,Liczba pięter w budynku,Rok budowy,Powierzchnia całkowita,Kuchnia,Informacje dodatkowe:,Okolica:,Dostępne od,Czynsz,Powierzchnia piwnicy
0,https://www.domiporta.pl/nieruchomosci/sprzeda...,154467824,agencja,23892,rw,Mieszkania na sprzedaż,"Warszawa, Śródmieście, Natolińska",650 000 zł,"19 420,38 zł/m2",kamienica,cegła,własność,1,496/9046/OMS,4,6,1959,"33,47 m2",z oknem,NaN,NaN,NaN,NaN,NaN
1,https://www.domiporta.pl/nieruchomosci/sprzeda...,154386831,agencja,23892,rw,Mieszkania na sprzedaż,"Warszawa, Praga-Południe, Łukowska",713 000 zł,"10 986,13 zł/m2",blok,cegła,spółdzielcze własnościowe,3,505/9046/OMS,2,3,1994,"64,90 m2",oddzielna,NaN,NaN,NaN,NaN,NaN
2,https://www.domiporta.pl/nieruchomosci/sprzeda...,154337833,agencja,23892,rw,Mieszkania na sprzedaż,"Warszawa, Mokotów, Stegny, Eukaliptusowa",1 740 000 zł,"19 456,56 zł/m2",blok,cegła,własność,4,500/9046/OMS,1,2,2022,"89,43 m2",otwarta,"winda, piwnica","szkoła, przedszkole",NaN,NaN,NaN
3,https://www.domiporta.pl/nieruchomosci/sprzeda...,154189330,agencja,23892,rw,Mieszkania na sprzedaż,"Warszawa, Wola, Mirów, Twarda",1 450 000 zł,"15 566,29 zł/m2",apartamentowiec,cegła,spółdzielcze własnościowe z kw,5,477/9046/OMS,6,9,1991,"93,15 m2",oddzielna,"winda, piwnica","szkoła, przedszkole, park",NaN,NaN,NaN
4,https://www.domiporta.pl/nieruchomosci/sprzeda...,153750120,agencja,23892,rw,Mieszkania na sprzedaż,"Warszawa, Mokotów, Bartycka",957 651 zł,6 900 zł/m2,inny,ytong,spółdzielcze własnościowe,7,369/9046/OMS,1,2,1994,"138,79 m2",oddzielna,NaN,NaN,NaN,NaN,NaN


In [202]:
dzielnice_warszawy = ['Bemowo',
'Białołęka',
'Bielany',
'Mokotów',
'Ochota',
'Praga Południe',
'Praga Północ',
'Rembertów',
'Śródmieście',
'Targówek',
'Ursus',
'Ursynów',
'Wawer',
'Wesoła',
'Wilanów',
'Włochy',
'Wola',
'Żoliborz',
'Praga-Południe',
'Praga-Północ']  # Lista wybranych dzielnic


def extract_dzielnica(row):
    for dzielnica in dzielnice_warszawy:
        if dzielnica in row['Lokalizacja']:
            return dzielnica
    return None

df['Dzielnica'] = df.apply(extract_dzielnica, axis=1)

In [203]:
# Funkcja do usunięcia słowa Warszawa z lokalizacji
def remove_word(row):
    return row.replace('Warszawa,', '').strip()

# Zastosowanie funkcji do kolumny
df['Lokalizacja'] = df['Lokalizacja'].apply(remove_word)

In [204]:
#usunięcie dzielnicy z Lokalizacji
def remove_dzielnica_from_lokalizacja(row):
    dzielnica = extract_dzielnica(row)
    if dzielnica:
        return row['Lokalizacja'].replace(dzielnica, '').strip()
    return row['Lokalizacja']

df['Lokalizacja'] = df.apply(remove_dzielnica_from_lokalizacja, axis=1)
df['Lokalizacja'] = df['Lokalizacja'].apply(lambda x: x.lstrip(', '))

In [205]:
#Tworzenie nowych kolumn 'Osiedle' i 'Ulica'
df['Osiedle'] = df['Lokalizacja'].apply(lambda x: x.split(',')[0].strip() if ',' in x else None)
df['Ulica'] = df['Lokalizacja'].apply(lambda x: x.split(',')[1].strip() if ',' in x else x)

In [206]:
df.drop('Lokalizacja', axis=1, inplace=True)

In [146]:
df

,url,advertId,advertiserType,advertiserId,market,Kategoria,Cena,Cena za m2,Typ budynku,Materiał,Forma własności,Liczba pokoi,Numer oferty,Piętro,Liczba pięter w budynku,Rok budowy,Powierzchnia całkowita,Kuchnia,Informacje dodatkowe:,Okolica:,Dostępne od,Czynsz,Powierzchnia piwnicy,Dzielnica,Osiedle,Ulica
0,https://www.domiporta.pl/nieruchomosci/sprzeda...,154467824,agencja,23892,rw,Mieszkania na sprzedaż,650 000 zł,"19 420,38 zł/m2",kamienica,cegła,własność,1,496/9046/OMS,4,6,1959,"33,47 m2",z oknem,NaN,NaN,NaN,NaN,NaN,Śródmieście,None,Natolińska
1,https://www.domiporta.pl/nieruchomosci/sprzeda...,154386831,agencja,23892,rw,Mieszkania na sprzedaż,713 000 zł,"10 986,13 zł/m2",blok,cegła,spółdzielcze własnościowe,3,505/9046/OMS,2,3,1994,"64,90 m2",oddzielna,NaN,NaN,NaN,NaN,NaN,Praga-Południe,None,Łukowska
2,https://www.domiporta.pl/nieruchomosci/sprzeda...,154337833,agencja,23892,rw,Mieszkania na sprzedaż,1 740 000 zł,"19 456,56 zł/m2",blok,cegła,własność,4,500/9046/OMS,1,2,2022,"89,43 m2",otwarta,"winda, piwnica","szkoła, przedszkole",NaN,NaN,NaN,Mokotów,Stegny,Eukaliptusowa
3,https://www.domiporta.pl/nieruchomosci/sprzeda...,154189330,agencja,23892,rw,Mieszkania na sprzedaż,1 450 000 zł,"15 566,29 zł/m2",apartamentowiec,cegła,spółdzielcze własnościowe z kw,5,477/9046/OMS,6,9,1991,"93,15 m2",oddzielna,"winda, piwnica","szkoła, przedszkole, park",NaN,NaN,NaN,Wola,Mirów,Twarda
4,https://www.domiporta.pl/nieruchomosci/sprzeda...,153750120,agencja,23892,rw,Mieszkania na sprzedaż,957 651 zł,6 900 zł/m2,inny,ytong,spółdzielcze własnościowe,7,369/9046/OMS,1,2,1994,"138,79 m2",oddzielna,NaN,NaN,NaN,NaN,NaN,Mokotów,None,Bartycka
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,https://www.domiporta.pl/nowe/ogloszenie/ogrod...,639126,deweloper,39035,rp,Mieszkania na sprzedaż,599 000 zł,7 987 zł/m2,NaN,cegła,NaN,NaN,4A,Parter,NaN,NaN,75 m2,NaN,"łącze internetowe, ochrona, telewizja kablowa,...","szkoła, park / las, przychodnia / szpital, cen...",NaN,NaN,NaN,Wawer,None,Tawułkowa 5
1072,https://www.domiporta.pl/nowe/ogloszenie/port_...,637613,deweloper,37869,rp,Mieszkania na sprzedaż,1 498 571 zł,21 525 zł/m2,NaN,pustak,NaN,3,Sierak1_19,1,NaN,2018,"69,62 m2",NaN,"plac zabaw, teren zamknięty, winda, ochrona, u...","szkoła, park / las, tramwaj, przychodnia / szp...",NaN,NaN,NaN,Praga Północ,None,Sierakowskiego/Okrzei
1073,https://www.domiporta.pl/nowe/ogloszenie/port_...,637615,deweloper,37869,rp,Mieszkania na sprzedaż,2 576 700 zł,26 250 zł/m2,NaN,pustak,NaN,3,Sierak1_21,1,NaN,2018,"98,16 m2",NaN,"plac zabaw, teren zamknięty, winda, ochrona, u...","szkoła, park / las, tramwaj, przychodnia / szp...",NaN,NaN,NaN,Praga Północ,None,Sierakowskiego/Okrzei
1074,https://www.domiporta.pl/nowe/ogloszenie/moj_u...,644166,deweloper,32372,rp,Mieszkania na sprzedaż,NaN,NaN,NaN,NaN,NaN,2,MU_5_U_7_5,7,NaN,2024,"48,13 m2",NaN,NaN,NaN,NaN,NaN,NaN,Ursus,None,ul. Posąg 7 Panien


In [257]:
df2.to_csv('domiporta_warszawa_30_str.csv', index=False)

In [56]:
df = pd.read_csv('domiporta_warszawa_30_str.csv', encoding='utf-8')


In [207]:
filtr = ~df['Cena za m2'].astype(str).str.endswith('zł/m2') & ~df['Cena za m2'].isna()
df = df.drop(336)

In [178]:
df[filtr]

/var/folders/75/8ybrj1sn4rv_ccfth6dy97hw0000gn/T/ipykernel_23493/260469309.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[filtr]


,url,advertId,advertiserType,advertiserId,market,Kategoria,Cena,Cena za m2,Typ budynku,Materiał,Forma własności,Liczba pokoi,Numer oferty,Piętro,Liczba pięter w budynku,Rok budowy,Powierzchnia całkowita,Kuchnia,Informacje dodatkowe:,Okolica:,Dostępne od,Czynsz,Powierzchnia piwnicy,Dzielnica,Osiedle,Ulica


In [208]:
df = df.drop('Okolica:', axis=1)

In [209]:
#Funkcja do wyciągnięcia Ceny ze stringa
def extract_number(text):
    # Używamy wyrażenia regularnego, aby znaleźć wszystkie cyfry w tekście
    numbers = re.findall(r'\d+', text)
    # Jeśli znaleziono cyfry, łączymy je w jedną liczbę (bez spacji) i przekształcamy na float
    if numbers:
        return float(''.join(numbers))
    else:
        return None

In [210]:
#Zmiana ze stringa na float
df['Cena'] = df['Cena'].apply(lambda x: extract_number(x) if pd.notna(x) else x)

In [211]:
#zmiana m2 na float
df['Cena za m2'] = df['Cena za m2'].str.replace(' zł/m2', '', regex=True).str.replace(' ', '', regex=True).str.replace(',', '.', regex=True).astype(float)

In [212]:
#Powierzchnia za m2
df['Powierzchnia piwnicy'] = df['Powierzchnia piwnicy'].str.replace(' m2', '', regex=True).str.replace(' ', '', regex=True).str.replace(',', '.', regex=True).astype(float)

In [213]:
#Robienie dummies z danych
dummies_informacje_dodatkowe = df['Informacje dodatkowe:'].str.get_dummies(sep=',')
df = pd.concat([df.drop(['Informacje dodatkowe:'], axis=1), dummies_informacje_dodatkowe, ], axis=1)



In [217]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1075 entries, 0 to 1075
Data columns (total 58 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   url                      1075 non-null   object 
 1   advertId                 1075 non-null   object 
 2   advertiserType           1075 non-null   object 
 3   advertiserId             1075 non-null   object 
 4   market                   1075 non-null   object 
 5   Kategoria                1075 non-null   object 
 6   Cena                     900 non-null    float64
 7   Cena za m2               897 non-null    float64
 8   Typ budynku              750 non-null    object 
 9   Materiał                 916 non-null    object 
 10  Forma własności          365 non-null    object 
 11  Liczba pokoi             1048 non-null   object 
 12  Numer oferty             997 non-null    object 
 13  Piętro                   1054 non-null   object 
 14  Liczba pięter w budynku 

In [239]:
df2 = df.drop([' dojazd asfaltowy',' domofon / wideofon',' fitness club', ' garderoba', ' komórka lokatorska', ' po remoncie',  ' prysznic', ' telefon', ' telewizja kablowa', ' usługi w budynku', ' wanna', ' łącze internetowe',' fitness club', ' ogródek', 'plac zabaw',], axis=1)

In [246]:
df2

,url,advertId,advertiserType,advertiserId,market_type,Kategoria,price,row_price_m2,building_type,material,ownership_form,no_rooms,Numer oferty,property_level,total_property_level,building_year,total_area,kitchen_type,Dostępne od,property_rent,Powierzchnia piwnicy,district,Osiedle,street,balkon,garaż,garaż podziemny,loggia,miejsce parkingowe,miejsce postojowe,ochrona,ogrzewanie centralne,ogrzewanie miejskie,ogrzewanie podłogowe,seperate_wc,parking strzeżony,piwnica,taras,teren strzeżony,teren zamknięty,widna kuchnia,elevator,balkon,fitness club,garaż,garaż podziemny,komórka lokatorska,loggia,miejsce parkingowe,miejsce postojowe,ogrzewanie centralne,ogrzewanie miejskie,ogródek,osobne wc,basement,teren zamknięty,winda,łącze internetowe,portal_name,city
0,https://www.domiporta.pl/nieruchomosci/sprzeda...,154467824,agencja,23892,rw,Mieszkania na sprzedaż,650000.0,19420.38,kamienica,cegła,własność,1,496/9046/OMS,4,6,1959,"33,47 m2",z oknem,NaN,NaN,NaN,Śródmieście,None,Natolińska,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,domiporta,Warszawa
1,https://www.domiporta.pl/nieruchomosci/sprzeda...,154386831,agencja,23892,rw,Mieszkania na sprzedaż,713000.0,10986.13,blok,cegła,spółdzielcze własnościowe,3,505/9046/OMS,2,3,1994,"64,90 m2",oddzielna,NaN,NaN,NaN,Praga-Południe,None,Łukowska,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,domiporta,Warszawa
2,https://www.domiporta.pl/nieruchomosci/sprzeda...,154337833,agencja,23892,rw,Mieszkania na sprzedaż,1740000.0,19456.56,blok,cegła,własność,4,500/9046/OMS,1,2,2022,"89,43 m2",otwarta,NaN,NaN,NaN,Mokotów,Stegny,Eukaliptusowa,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,domiporta,Warszawa
3,https://www.domiporta.pl/nieruchomosci/sprzeda...,154189330,agencja,23892,rw,Mieszkania na sprzedaż,1450000.0,15566.29,apartamentowiec,cegła,spółdzielcze własnościowe z kw,5,477/9046/OMS,6,9,1991,"93,15 m2",oddzielna,NaN,NaN,NaN,Wola,Mirów,Twarda,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,domiporta,Warszawa
4,https://www.domiporta.pl/nieruchomosci/sprzeda...,153750120,agencja,23892,rw,Mieszkania na sprzedaż,957651.0,6900.00,inny,ytong,spółdzielcze własnościowe,7,369/9046/OMS,1,2,1994,"138,79 m2",oddzielna,NaN,NaN,NaN,Mokotów,None,Bartycka,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,domiporta,Warszawa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,https://www.domiporta.pl/nowe/ogloszenie/ogrod...,639126,deweloper,39035,rp,Mieszkania na sprzedaż,599000.0,7987.00,NaN,cegła,NaN,NaN,4A,Parter,NaN,NaN,75 m2,NaN,NaN,NaN,NaN,Wawer,None,Tawułkowa 5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,domiporta,Warszawa
1072,https://www.domiporta.pl/nowe/ogloszenie/port_...,637613,deweloper,37869,rp,Mieszkania na sprzedaż,1498571.0,21525.00,NaN,pustak,NaN,3,Sierak1_19,1,NaN,2018,"69,62 m2",NaN,NaN,NaN,NaN,Praga Północ,None,Sierakowskiego/Okrzei,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,domiporta,Warszawa
1073,https://www.domiporta.pl/nowe/ogloszenie/port_...,637615,deweloper,37869,rp,Mieszkania na sprzedaż,2576700.0,26250.00,NaN,pustak,NaN,3,Sierak1_21,1,NaN,2018,"98,16 m2",NaN,NaN,NaN,NaN,Praga Północ,None,Sierakowskiego/Okrzei,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,domiporta,Warszawa
1074,https://www.domiporta.pl/nowe/ogloszenie/moj_u...,644166,deweloper,32372,rp,Mieszkania na sprzedaż,NaN,NaN,NaN,NaN,NaN,2,MU_5_U_7_5,7,NaN,2024,"48,13 m2",NaN,NaN,NaN,NaN,Ursus,None,ul. Posąg 7 Panien,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,domiporta,Warszawa


In [241]:
df2 = df2.assign(portal_name='domiporta')

In [242]:
df2 = df2.assign(city = 'Warszawa') 

In [243]:
df2 = df2.rename(columns={'Typ budynku': 'property_type', 'Cena':'price', 'Cena za m2': 'row_price_m2', 'Dzielnica':'district','Ulica':'street', 'Powierzchnia całkowita':'total_area', 'Piętro':'property_level', 'Liczba pięter w budynku':'total_property_level', 'Liczba pokoi':'no_rooms', 'Kuchnia':'kitchen_type', 'Forma własności':'ownership_form', 'Typ budynku':'building_type','Rok budowy':'building_year', 'Materiał':'material', ' winda':' elevator','Czynsz':'property_rent', 'piwnica':'basement', 'market':'market_type', ' osobne wc':'seperate_wc'   })

In [251]:
new_column_order = ['url', 'portal_name', 'market_type', 'price', 'row_price_m2', 'city', 'district', 'street', 'total_area', 'property_level', 'total_property_level', 'no_rooms', 'kitchen_type', 'ownership_form', 'building_type', 'building_year', 'material','basement']

In [252]:
remaining_columns = [col for col in df2.columns if col not in new_column_order]
df2 = df2[new_column_order + remaining_columns]

In [253]:
df2.head(5)

,url,portal_name,market_type,price,row_price_m2,city,district,street,total_area,property_level,total_property_level,no_rooms,kitchen_type,ownership_form,building_type,building_year,material,basement,advertId,advertiserType,advertiserId,Kategoria,Numer oferty,Dostępne od,property_rent,Powierzchnia piwnicy,Osiedle,balkon,garaż,garaż podziemny,loggia,miejsce parkingowe,miejsce postojowe,ochrona,ogrzewanie centralne,ogrzewanie miejskie,ogrzewanie podłogowe,seperate_wc,parking strzeżony,piwnica,taras,teren strzeżony,teren zamknięty,widna kuchnia,elevator,balkon,fitness club,garaż,garaż podziemny,komórka lokatorska,loggia,miejsce parkingowe,miejsce postojowe,ogrzewanie centralne,ogrzewanie miejskie,ogródek,osobne wc,teren zamknięty,winda,łącze internetowe
0,https://www.domiporta.pl/nieruchomosci/sprzeda...,domiporta,rw,650000.0,19420.38,Warszawa,Śródmieście,Natolińska,"33,47 m2",4,6,1,z oknem,własność,kamienica,1959,cegła,0,154467824,agencja,23892,Mieszkania na sprzedaż,496/9046/OMS,NaN,NaN,NaN,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,https://www.domiporta.pl/nieruchomosci/sprzeda...,domiporta,rw,713000.0,10986.13,Warszawa,Praga-Południe,Łukowska,"64,90 m2",2,3,3,oddzielna,spółdzielcze własnościowe,blok,1994,cegła,0,154386831,agencja,23892,Mieszkania na sprzedaż,505/9046/OMS,NaN,NaN,NaN,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,https://www.domiporta.pl/nieruchomosci/sprzeda...,domiporta,rw,1740000.0,19456.56,Warszawa,Mokotów,Eukaliptusowa,"89,43 m2",1,2,4,otwarta,własność,blok,2022,cegła,0,154337833,agencja,23892,Mieszkania na sprzedaż,500/9046/OMS,NaN,NaN,NaN,Stegny,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,https://www.domiporta.pl/nieruchomosci/sprzeda...,domiporta,rw,1450000.0,15566.29,Warszawa,Wola,Twarda,"93,15 m2",6,9,5,oddzielna,spółdzielcze własnościowe z kw,apartamentowiec,1991,cegła,0,154189330,agencja,23892,Mieszkania na sprzedaż,477/9046/OMS,NaN,NaN,NaN,Mirów,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,https://www.domiporta.pl/nieruchomosci/sprzeda...,domiporta,rw,957651.0,6900.00,Warszawa,Mokotów,Bartycka,"138,79 m2",1,2,7,oddzielna,spółdzielcze własnościowe,inny,1994,ytong,0,153750120,agencja,23892,Mieszkania na sprzedaż,369/9046/OMS,NaN,NaN,NaN,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [107]:
#zmiana m2 na float
df['Powierzchnia całkowita'] = df['Powierzchnia całkowita'].str.replace(' m2', '', regex=True).str.replace(',', '.', regex=True).astype(float)
#zmiana zl na float
df['Czynsz'] = df['Czynsz'].str.replace(' PLN', '', regex=True).str.replace(' ', '', regex=True).astype(float)
#Cena za m2
df['Cena za m2'] = df['Cena za m2'].str.replace(' zł/m2', '', regex=True).str.replace(' ', '', regex=True).str.replace(',', '.', regex=True).astype(float)
#Powierzchnia za m2
df['Powierzchnia piwnicy'] = df['Powierzchnia piwnicy'].str.replace(' m2', '', regex=True).str.replace(' ', '', regex=True).str.replace(',', '.', regex=True).astype(float)


ValueError: could not convert string to float: '5177.11EUR(23297.00)'

In [2]:
#Funkcja do wyciągnięcia Ceny ze stringa
def extract_number(text):
    # Używamy wyrażenia regularnego, aby znaleźć wszystkie cyfry w tekście
    numbers = re.findall(r'\d+', text)
    # Jeśli znaleziono cyfry, łączymy je w jedną liczbę (bez spacji) i przekształcamy na float
    if numbers:
        return float(''.join(numbers))
    else:
        return None

In [3]:
#Zmiana ze stringa na float
df['Cena'] = df['Cena'].apply(lambda x: extract_number(x) if pd.notna(x) else x)

NameError: name 'df' is not defined

In [123]:
#Robienie dummies z danych
dummies_informacje_dodatkowe = df['Informacje dodatkowe:'].str.get_dummies(sep=',')
dummies_okolica = df['Okolica:'].str.get_dummies(sep=',')
dummies_typ_budynku = df['Typ budynku'].str.get_dummies(sep=',')
dummies_forma_wlasnosci = df['Forma własności'].str.get_dummies()
dummies_kuchnia = df['Kuchnia'].str.get_dummies()
dummies_dzielnica = df['Dzielnica'].str.get_dummies()
dummies_material = df['Materiał'].str.get_dummies()
df = pd.concat([df.drop(['Informacje dodatkowe:','Okolica:','Typ budynku','Forma własności','Kuchnia','Dzielnica','Materiał'], axis=1), dummies_informacje_dodatkowe, dummies_okolica, dummies_typ_budynku, dummies_forma_wlasnosci, dummies_kuchnia, dummies_dzielnica, dummies_material ], axis=1)




In [124]:
#Zmiana z parter na 0
df['Piętro'] = df['Piętro'].replace('Parter', 0)


In [231]:
df['Liczba pokoi'] = df['Liczba pokoi'].fillna(0)  # Zastąp NaN zerem
df['Piętro'] = df['Piętro'].fillna(0)  # Zastąp NaN zerem
df['Rok budowy'] = df['Rok budowy'].fillna(0)  # Zastąp NaN zerem
df['Liczba pięter w budynku'] = df['Liczba pięter w budynku'].fillna(0)  # Zastąp NaN zerem

In [232]:


# Zamiana kolumny 'Liczba pokoi' na int
df['Liczba pokoi'] = df['Liczba pokoi'].astype(int)

# Zamiana kolumny 'Piętro' na int
df['Piętro'] = df['Piętro'].astype(int)

# Zamiana kolumny 'Rok budowy' na int
df['Rok budowy'] = df['Rok budowy'].astype(int)

# Zamiana kolumny 'Rok budowy' na int
df['Liczba pięter w budynku'] = df['Liczba pięter w budynku'].astype(int)

In [233]:
df

,Kategoria,Liczba pokoi,Numer oferty,Piętro,Rok budowy,Powierzchnia całkowita,Cena,Cena za m2,Dostępne od,Liczba pięter w budynku,Czynsz,Powierzchnia piwnicy,Osiedle,Ulica,balkon,dojazd asfaltowy,fitness club,garaż,loggia,miejsce parkingowe,miejsce postojowe,ochrona,ogrzewanie centralne,ogrzewanie miejskie,ogródek,osobne wc,piwnica,prysznic,telefon,telewizja kablowa,teren strzeżony,teren zamknięty,usługi w budynku,wanna,winda,łącze internetowe,balkon,fitness club,garaż podziemny,miejsce parkingowe,ogrzewanie miejskie,piwnica,plac zabaw,teren zamknięty,winda,łącze internetowe,apteka,autobus,bazarek osiedlowy,centrum handlowe,dworzec kolejowy,kolej podmiejska,las,metro,niska zabudowa,park,park / las,pkp,plac zabaw,poczta,policja,przedszkole,przychodnia,przychodnia / szpital,sklep spożywczy,szkoła,tramwaj,żłobek,apteka,autobus,metro,niska zabudowa,park,poczta,przychodnia,szkoła,apartamentowiec,blok,dom wielorodzinny,inny,kamienica,Odrębna własność lokalu,SPÓŁDZIELCZE WŁASNOŚCIOWE Z KW,Spółdzielcze własnościowe,Własność,hipoteczne,własność,aneks,aneks kuchenny,aneks kuchenny - połączony z salonem,jasna z oknem,oddzielna,otwarta,półotwarta,widna,z oknem,Bemowo,Białołęka,Bielany,Mokotów,Ochota,Praga Południe,Praga Północ,Praga-Południe,Praga-Północ,Targówek,Ursus,Ursynów,Wawer,Wesoła,Wilanów,Wola,Włochy,Śródmieście,Żoliborz,beton,cegła,inny,pustak,rama h,wielka płyta
0,Mieszkania na sprzedaż,3,I.1.02,0,2024,70.75,NaN,NaN,NaN,0,NaN,NaN,None,Redutowa 9,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,0,1,1,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,1,1,0,1,0,1,1,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,Mieszkania na sprzedaż,4,D0302,3,2024,122.11,2341000.0,19171.00,NaN,0,NaN,NaN,None,ul. Rybałtów róg Wilczy Dół,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Mieszkania na sprzedaż,2,A0002,0,2024,55.20,1081200.0,19587.00,NaN,0,NaN,NaN,None,ul. Rybałtów róg Wilczy Dół,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Mieszkania na sprzedaż,2,NaN,1,2014,54.20,921000.0,16993.00,1.05.2023r.,6,614.0,NaN,None,Sokratesa 9,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,0,1,1,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Mieszkania na sprzedaż,3,1603/1631/OMS,3,1954,59.00,1390000.0,23559.32,4.10.2023r.,6,NaN,NaN,Oleandrów,pl. Plac Konstytucji,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Mieszkania na sprzedaż,3,NaN,2,2005,78.00,1063800.0,13638.46,NaN,7,NaN,NaN,None,ul. Puławska,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
140,Mieszkania na sprzedaż,3,BECI121,6,1980,62.00,715000.0,11532.26,NaN,10,NaN,NaN,None,Bora-Komorowskiego,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [234]:
df2 = df.drop(['Kategoria','Numer oferty','Dostępne od', 'Ulica','Osiedle'], axis=1)

In [205]:
pd.set_option('display.max_columns', None)